In [0]:
dbutils.fs.ls('mnt/silver/SalesLT')

[FileInfo(path='dbfs:/mnt/silver/SalesLT/Address/', name='Address/', size=0, modificationTime=1726070313000),
 FileInfo(path='dbfs:/mnt/silver/SalesLT/Customer/', name='Customer/', size=0, modificationTime=1726070332000),
 FileInfo(path='dbfs:/mnt/silver/SalesLT/CustomerAddress/', name='CustomerAddress/', size=0, modificationTime=1726070748000),
 FileInfo(path='dbfs:/mnt/silver/SalesLT/Product/', name='Product/', size=0, modificationTime=1726070758000),
 FileInfo(path='dbfs:/mnt/silver/SalesLT/ProductCategory/', name='ProductCategory/', size=0, modificationTime=1726070768000),
 FileInfo(path='dbfs:/mnt/silver/SalesLT/ProductDescription/', name='ProductDescription/', size=0, modificationTime=1726070777000),
 FileInfo(path='dbfs:/mnt/silver/SalesLT/ProductModel/', name='ProductModel/', size=0, modificationTime=1726070786000),
 FileInfo(path='dbfs:/mnt/silver/SalesLT/ProductModelProductDescription/', name='ProductModelProductDescription/', size=0, modificationTime=1726070795000),
 FileInf

In [0]:
dbutils.fs.ls('mnt/gold/')

[FileInfo(path='dbfs:/mnt/gold/SalesLT/', name='SalesLT/', size=0, modificationTime=1726070515000)]

In [0]:
input_path = 'dbfs:/mnt/silver/SalesLT/ '

In [0]:
# listing tables
tables_name = []
for i in dbutils.fs.ls('dbfs:/mnt/silver/SalesLT/'):
    tables_name.append(i.name.split('/')[0])


In [0]:
tables_name

['Address',
 'Customer',
 'CustomerAddress',
 'Product',
 'ProductCategory',
 'ProductDescription',
 'ProductModel',
 'ProductModelProductDescription',
 'SalesOrderDetail',
 'SalesOrderHeader']

DOING TRANFORMATION FOR ALL TABLES (CHANGING THE COLUMN NAMES)

In [0]:
for name in tables_name:
    # Construct the path for reading the Delta table
    path = f'/mnt/silver/SalesLT/{name}'

    # Load the DataFrame
    df = spark.read.format('delta').load(path)

    # Get the list of old column names
    old_columns = df.columns

    # Create a list to store transformed column names
    new_columns = []

    # Rename columns from CamelCase to snake_case
    for col in old_columns:
        new_col = col[0]  # Start with the first character

        # Iterate over the rest of the characters
        for char in col[1:]:
            if char.isupper():
                new_col += '_'  # Insert an underscore before uppercase letters
                new_col += char.lower()  # Convert uppercase letter to lowercase
            else:
                new_col += char

        # Check for duplicate column names and skip if necessary
        if new_col in new_columns:
            print(f"Skipping column {col} as it results in a duplicate name '{new_col}'")
            new_columns.append(col)  # Keep the old name if duplicate found
        else:
            new_columns.append(new_col)

    # Rename all columns in one step using toDF
    df = df.toDF(*new_columns)

    # Define the output path for saving the transformed DataFrame
    output_path = f'/mnt/gold/SalesLT/{name}/'

    # Write the DataFrame to Delta format
    df.write.format('delta').mode('overwrite').save(output_path)

